In [1]:
import pandas as pd

data = pd.read_csv('/kaggle/input/train-test-val/test.csv')
data.head(5)

,image_id,image_path,question,response
0,61BorBlKJmL,images/small/c3/c34ecbb9.jpg,What is the color of the phone case?,Orange
1,81Ba3iri9eL,images/small/56/565b61e4.jpg,"What is the heel's measurement, in inches?",One
2,416CFhHPU6L,images/small/8b/8b497642.jpg,Is the cup designed for cold beverages?,No
3,41VkOGAbdAL,images/small/66/666e46a0.jpg,Where was it made?,China
4,61Ad3nx6IAL,images/small/c0/c089d4ac.jpg,What phone models is the case for?,M21


In [2]:
from transformers import BlipProcessor, BlipForQuestionAnswering
from PIL import Image
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xl", torch_dtype=torch.float16)

# processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16)
path = "/kaggle/input/weights-67k/kaggle/working/weights"
processor = BlipProcessor.from_pretrained(path)
model = BlipForQuestionAnswering.from_pretrained(path, torch_dtype=torch.float16)

model.to(device)
model.eval()

2025-05-16 01:34:39.681999: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747359279.866874      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747359279.919908      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

BlipForQuestionAnswering(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-11): 12 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((768,), eps=1e-05, e

In [3]:
def help(image_path,question):
  try:
      image = Image.open(image_path).convert("RGB")
  except:
      return "error"
  # prompt = f"Question:{question} Answer:"
  inputs = processor(images=image, text=question, return_tensors="pt").to(device, torch.float16)
  generated_ids = model.generate(**inputs,max_new_tokens=10,use_cache=True)
  generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
  return generated_text

In [4]:
import re
predictions = []
c=0
for idx, row in data.iterrows():
    image_path = str(row['image_path'])
    question = str(row['question'])
    raw_answer = help("/kaggle/input/vrdataset/"+image_path, question)
    predictions.append(raw_answer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [5]:
print(predictions)

['orange', 'one', 'no', 'india', 'samsung', 'blue', 'polypropyne', 'black', 'lentil', 'lace', 'oriental', 'synthetic', 'cheese', 'a3s', 'yes', 'no', 'almonds', 'camera', 'hearts', 'three', 'no', 'fitness', 'wholefoods', 'yes', 'vivo', 'five', 'no', 'led', 'wholefoods', 'no', 'stylish', 'two', 'brown', 'snug', 'yes', 'protection', 'hard', 'no', 'queen', 'no', 'fabric', 'strap', 'three', 'yes', 'metal', 'stylish', 'rectangular', 'diamonds', 'yes', 'polyester', 'stylish', 'back', 'suede', 'portugal', 'wool', 'lg', 'polyester', 'pvc', 'samsung', 'hard', 'geometric', 'nokia', 'block', 'amazon', 'yes', 'gold', 'basic', 'wholefoods', 'twelve', 'aluminum', 'three', 'fifteen', 'wood', 'polyester', 'excrement', 'flower', 'stylish', 'three', 'grade', 'lace', 'copper', 'j2', 'usa', 'tpu', 'ceramic', 'round', 'embossed', 'right', 'sixty', 'amazonbasics', 'yes', 'ceiling', 'yes', 'garlic', 'orange', 'three', 'two', 'india', 'thirty', 'nokia', 'presciutto', 'round', 'beige', 'samsung', 'phone', 'yes'

In [9]:
empty_count = predictions.count('')
print("Number of empty strings:", empty_count)

Number of empty strings: 1


In [10]:
!pip install bert-score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-10.3

In [11]:
from bert_score import score

references = data['response'].astype(str).tolist()

P, R, F1 = score(predictions, references, lang="en")

# Average F1
print(f"Mean BERTScore F1: {F1.mean().item():.4f}")
print(f"Mean BERTScore Precision: {P.mean().item():.4f}")
print(f"Mean BERTScore Recall   : {R.mean().item():.4f}")

# ------- Base-Line ------
# Mean BERTScore F1: 0.8933
# Mean BERTScore Precision: 0.8859
# Mean BERTScore Recall   : 0.9030


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Mean BERTScore F1: 0.9386
Mean BERTScore Precision: 0.9456
Mean BERTScore Recall   : 0.9337


In [12]:
def normalize(text):
    return str(text).strip().lower()

norms_pred = predictions.copy()

# Apply normalization
for i in norms_pred:
  i = normalize(i)
normalized_truths = data['response'].apply(normalize)

# Compare for exact match
k = norms_pred == normalized_truths

# Compute accuracy
exact_match_accuracy = k.mean()
print(f"Exact Match Accuracy: {exact_match_accuracy:.5f}")

# Exact Match Accuracy: 0.19072 - BaseLine -----

Exact Match Accuracy: 0.51816


In [13]:
!pip install sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
from sentence_transformers import SentenceTransformer, util

# Load a lightweight and suitable model for short text
model = SentenceTransformer('all-MiniLM-L6-v2')

# Convert to embeddings
pred_embeds = model.encode(predictions, convert_to_tensor=True)
ref_embeds = model.encode(references, convert_to_tensor=True)

# Compute cosine similarities for each pair
cosine_scores = util.cos_sim(pred_embeds, ref_embeds)

# Average score (diagonal contains the matching prediction-reference pairs)
average_cosine_score = cosine_scores.diag().mean().item()
print(f"Average Cosine Similarity Score: {average_cosine_score:.4f}")

# Average Cosine Similarity Score: 0.515 -- BaseLine

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/782 [00:00<?, ?it/s]

Batches:   0%|          | 0/782 [00:00<?, ?it/s]

Average Cosine Similarity Score: 0.7275


In [15]:
# Cosine similarity as pass/fail
correct = (cosine_scores.diag() >= 0.75).sum().item()
accuracy = correct / len(predictions)
accuracy

# BaseLine : 0.24336 - Best Consine Match

0.5578